# CASE STUDY - RAZOR GROUP
## PYTHON PROJECT

**Submitted by *Pratyusha Malla***

*The problem statement is to sanitize and analyze a dataset consisting of information pertaining to certain sellers on Amazon. The objective is to identify the best sellers among them who can be acquired.*

### Table of Contents

#### I. Description

[Summary of results](#summary)

[Methodology of analysis](#methodology)

#### II. Code

[Load data](#load_data)

[Remove Chinese sellers](#chinese_sellers)

[Create lists of column data](#col_lists)

[Parse to get the number of products](#product_count)

[Parse to get the percentage of positive ratings and number of ratings](#ratings)

[Parse to get the email address](#email)

[Parse to get the phone number](#phone)

[Parse to get the business name](#business_name)

[Analyze the sanitized data](#analysis)

[Display results](#results)

<a id='summary'></a>
#### Summary of results

The analysis returns 10 of the most promising sellers:

* Vinfiz GmbH
* Geschenke Direkt GmbH
* Zoreno Versandhandel GmbH
* Kjells Warenhandels GmbH
* Wallario GmbH
* Kessler electronic GmbH
* KS Licht- u. Elektrotechnik GmbH
* Tim Jochen Forster
* tasko Products GmbH
* VBS Hobby Service GmbH

The business addresses, contact numbers and email addresses of these sellers, if available, are displayed along with the rest of the details at the end of this notebook. See [results](#results).
In addition to displaying the results in the notebook, the code, when run, saves these details in a .csv file.

<a id='methodology'></a>
#### Methodology of analysis and selection

The analysis of the sellers is based on five key parameters as listed below:

1. Number of ratings available
2. Percentage of positive ratings
3. Number of products offered
4. Temporal trends of negative ratings
5. Number of ratings of hero products

For the first three parameters, the statistic used to filter data was the greater of the mean and median. The median is usually used when there is a high skewedness in the data. However, since the current context is that of a market and the aim is to find the best seller, the mean has been used if it is greater than the median even when the data is skewed. The reason is that in a competitive market, if seller A has a million products as opposed to 2000 others who have just 10 products each, the idea would be to select seller A.

The data thus filtered was sorted on the basis of the percentage of positive ratings and the number of ratings that the top two products of the seller have, in descending order, and the top 10 of these were selected as being eligible for acquisition. This list of the top 10 sellers consisted of a good mix of factors based on which a seller could be said to be promising - some had a good number of products, some others had a high number of ratings and so on.

Further filtering can be done using the columns improv_0 and improv_1 to see if the temporal trends of the seller are commendable (i.e. if the maximum percentage of negative ratings has decreased with time). However, for the top ten sellers obtained, further filtering using this criterion did not shed light on new, useful information and hence, this criterion was not used for selection.


<a id='load_data'></a>

In [1]:
#loading data

import pandas, re, unicodedata

df = pandas.read_csv('dataset.csv')

# dropping columns which give no useful information

for col in df.columns[:5]:
    df.drop(col, inplace = True, axis = 1)


<a id='chinese_sellers'></a>

In [2]:
#dropping rows containing data of Chinese sellers

address = list(df.loc[:,'businessaddress'])

for i in range(len(df)):
    
    # checking the last two characters of the businessaddress column to see if they form 'CN'
    
    if (str(address[i])[-2].lower() + str(address[i])[-1].lower()) == 'cn':
        df.drop(i, axis = 0, inplace = True)

# re-assigning indices after dropping the columns

df.index = range(len(df))


<a id='col_lists'></a>

In [3]:
# getting lists of column data

prodCount = list(df.loc[:,'sellerproductcount'])
sellerRating = list(df.loc[:,'sellerratings'])
contact = list(df.loc[:,'sellerdetails'])
names = list(df.loc[:,'seller business name'])


<a id='product_count'></a>

In [4]:
# getting the count of products the seller has

for i in range(len(df)):
    
    # as long as product count is a valid number and not an empty string, getting count from it
    
    if str(prodCount[i]) != 'nan' or '':
        
        # creating spaces at delimiting points for easier segregation
        
        j = str(prodCount[i]).replace(' results', '').replace(',', '')
        k = j.rfind(' ') + 1
        df.at[i,'sellerproductcount'] = int(j[k:])
        
    # if not, assigning 0
    
    else:
        df.at[i,'sellerproductcount'] = 0


<a id='ratings'></a>

In [5]:
# getting the % rating of the seller and no of ratings

noOfRatings = []
        
for i in range(len(df)):
    
    # as long as the seller ratings are positive, getting the % and total number
    
    if 'positive' in str(sellerRating[i]):
        
        # finding positions in the substring where required values exist
        
        j = sellerRating[i].find('%')
        k = sellerRating[i].find('(')
        l = sellerRating[i].find(')')
        phrase = sellerRating[i][k+1:l]
        m = phrase.find(' ')
        noOfRatings.append(int(phrase[:m]))
        
        # updating the value in the dataframe
        
        df.at[i,'sellerratings'] = float(sellerRating[i][:j])
    
    # if not, assigning 0
    
    else:
        noOfRatings.append(0)
        df.at[i,'sellerratings'] = 0
        
# creating a new column for the number of ratings

df['Ratings'] = noOfRatings


<a id='email'></a>

In [6]:
# getting the email address of the seller

email = []

for i in range(len(df)):
    p = str(contact[i]).lower().replace(':', ': ').replace('(', '( ').replace(')', ') ').replace(';', ' ; ')
    q = p.replace('.com', '.com ').replace('.de', '.de ')
    
    # finding the @ symbol and allowed characters around it using the re module offered by python
    
    id = re.findall(r'[\w\.-]+@[\w\.-]+', q)
    email.append(id)

# creating a new column for the email addresses of the sellers

df['EmailID'] = email


<a id='phone'></a>

In [7]:
# getting the phone number of the seller

telephone = []

# creating a list of characters allowed to be in the phone number

digits = [str(i) for i in range(10)]
allowed = digits
allowed.extend([' ', '-', '/', '(', ')', '.', ',', '|', '+'])

for i in range(len(df)):
    
    # removing unicode data
    
    p = str(contact[i]).replace('\n', '')
    p = unicodedata.normalize("NFKD", p)
    
    # finding the phrase 'tel' in the string
    
    a = p.lower().find('tel')
    
    # if the phrase is found
    
    if a != -1:
        
        # getting the substring containing numbers after the phrase tel is encountered
        
        q = p[a:]
        q = q.replace(':', ': ').replace('(', '( ').replace(')', ') ').replace(';', ' ; ')
        b = q.find(' ')
        r = q[b:]
        ind = 0
        
        # finding the index when a disallowed character is encountered for the first time
        
        for char in r:
            
            res = any(val == char for val in allowed)
            if not res:
                break
            else:
                ind += 1
        s = r[:ind]
        
        # cleaning up the string
        
        s = s.replace(' ', '').replace(',', '').replace('.', '').replace('|', '')
        if '(' in s and ')' not in s:
            s = s[:s.find('(')]

    else:
        
        # finding the phrase '+' in the string
        
        a = p.lower().find('+')
        
        # if the phrase is found
        
        if a != -1:
            
            # getting the substring containing numbers after the phrase tel is encountered for the first time

            q = p[a:]
            q = q.replace(':', ': ').replace('(', '( ').replace(')', ') ').replace(';', ' ; ').replace('+', '+ ')
            b = q.find(' ')
            r = q[b:]
            ind = 0
            
            # finding the index when a disallowed character is encountered

            for char in r:
                
                res = any(val == char for val in allowed)
                if not res:
                    break
                else:
                    ind += 1            
            s = r[:ind]
            
            # cleaning up the string
            
            s = s.replace(' ', '').replace(',', '').replace('.', '').replace('|', '')
            if ('(' in s) and (')' not in s):
                s = s[:s.find('(')]
        else:
            s = 0
    
    telephone.append(s)
    
# creating a new column for the phone numbers of the sellers
    
df['Number'] = telephone


<a id='business_name'></a>

In [8]:
# getting the business name

for i in range(len(df)):
    df.at[i,'seller business name'] = str(names[i]).replace('Business Name:', '')


In [9]:
# dropping redundant columns

df.drop(['sellerdetails'], inplace = True, axis = 1)

# creating a copy of the dataframe for analysis and dropping unnecessary columns from the copy

df1 = df.copy()
df1.drop(['seller business name', 'businessaddress', 'Sample brand name', 'Sample Brand URL'], inplace = True, axis = 1)


<a id='analysis'></a>

In [10]:
# analysing cleaned data

# creating new lists to see if the negative ratings have been decreasing with time
# using data from the 'maximum % of negative ratings' columns
# improv_0 is 1 if the max % of negative ratings has decreased from the last 90 days to the last 30 days
# improv_1 is 1 if the max % of negative ratings has decreased from the last 12 months to the last 90 days

improv_0 = []
improv_1 = []

for i in range(len(df1)):
    if df1.iloc[i][5] < df1.iloc[i][6]:
        improv_0.append(1)
        if df1.iloc[i][6] < df1.iloc[i][7]:
            improv_1.append(1)
        else:
            improv_1.append(0)
    else:
        improv_0.append(0)
        improv_1.append(0)

# creating the columns in the dataframe
        
df1['improv_0'] = improv_0
df1['improv_1'] = improv_1

# calculating statistics to filter sellers

l = df1['Ratings'].median()
m = df1['sellerratings'].median()
n = df1['sellerproductcount'].mean()

# filtering based on the statistics

df1 = df1.loc[df1['Ratings'] > l]
df1 = df1.loc[df1['sellerratings'] > m]
df1 = df1.loc[df1['sellerproductcount'] > n]

# sorting by the number of ratings of the hero products

df2 = df1.sort_values(['sellerratings', 'Hero Product 1 #ratings', 'Hero Product 2 #ratings'], ascending = False, inplace = False)

# retrieving the top 10 results

df3 = df2.iloc[0:10]

indices = list(df3.index)


<a id='results'></a>

In [11]:
# displaying the data for the top 10 sellers and saving them in a file

df.loc[indices].to_csv('top_sellers.csv')
df.loc[indices]


,sellerproductcount,sellerratings,seller business name,businessaddress,Count of seller brands,Max % of negative seller ratings - last 30 days,Max % of negative seller ratings - last 90 days,Max % of negative seller ratings - last 12 months,Hero Product 1 #ratings,Hero Product 2 #ratings,Sample brand name,Sample Brand URL,Ratings,EmailID,Number
440,6000,100.0,Vinfiz GmbH,Berliner Str.59GüterslohNordrhein-Westfalen333...,15,0,0,0,2436,716,STRICKER,https://www.amazon.de/-/en/STRICKER/b/ref=bl_d...,669,[info@vinfiz.de],05241/9977601
245,9000,99.0,Geschenke Direkt GmbH,Bornaische Str 26Hr WiechertMarkkleebergSachse...,16,3,1,1,10917,2527,Anzm,https://www.amazon.de/-/en/Anzm/b/ref=bl_dp_s_...,496,[],0314/25417150
594,100000,98.0,Zoreno Versandhandel GmbH,Blegistrasse 25BaarZG6340CH,16,3,2,2,25802,4979,(blank),(blank),2630,[ade.support@zoreno.com],0
72,7000,98.0,Kjells Warenhandels GmbH,Bänschstr. 63BerlinBerlin10247DE,15,6,3,2,7194,6529,Perel Garden,https://www.amazon.de/-/en/Perel-Garden/b/ref=...,200,[amazon@kjells.de],03054619947
480,20000,98.0,Wallario GmbH,Karl-Liebknecht-Str. 102CottbusBrandenburg03046DE,11,3,1,2,113,91,Visit the Wallario Store,https://www.amazon.de/-/en/stores/Wallario/pag...,497,[amazon@posterdepot.de],+49(0)35548692400
51,7000,97.0,Kessler electronic GmbH,Dieselstr.4Kerpen50170DE,16,3,3,2,47337,32420,Visit the Varta Store,https://www.amazon.de/-/en/stores/VARTA+/page/...,3416,[info@kessler-electronic.de],+49(0)2273/991-9325(
118,20000,97.0,KS Licht- u. Elektrotechnik GmbH,KS Licht- u. Elektrotechnik GmbHManderscheidts...,16,0,0,1,4889,1544,Visit the Paulmann Store,https://www.amazon.de/-/en/stores/Paulmann+Lic...,197,[service@ks-germany.com],+49-201-89924-0
311,50000,96.0,Tim Jochen Forster,Tim ForsterGartenstrasse 1-3Buch56357DE,16,0,7,4,18560,16960,Scottoiler,https://www.amazon.de/-/en/Scottoiler/b/ref=bl...,77,[amazon@tmf-racing.com],067729669046
500,10000,96.0,tasko Products GmbH,tasko Products GmbHMurrhardter Straße 8Backnan...,16,5,3,2,12487,3402,Visit the Victorinox Store,https://www.amazon.de/-/en/stores/Victorinox/p...,558,[amazon@welovebags.de],07191-3272-25
402,7000,96.0,VBS Hobby Service GmbH,Justus-von-Liebig-Str.8Verden27283DE,16,5,4,3,7102,2876,VBS,https://www.amazon.de/-/en/VBS/b/ref=bl_dp_s_w...,1681,[amazon@vbs-versand.de],0423166811
